In [ ]:

from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
import tensorflow.contrib.learn as skflow
from sklearn import svm, datasets
from sklearn.metrics import confusion_matrix
import shutil
import os

# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df,name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name,x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)

# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df,name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_

# Encode a numeric column as zscores
def encode_numeric_zscore(df,name,mean=None,sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name]-mean)/sd

# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)

# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df,target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    print(target_type)

    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        return df.as_matrix(result).astype(np.float32),df.as_matrix([target]).astype(np.int32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32),df.as_matrix([target]).astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

# Regression chart, we will see more of this chart in the next class.
def chart_regression(pred,y):
    t = pd.DataFrame({'pred' : pred.flatten(), 'y' : y_test.flatten()})
    t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()
    
def plot_confusion_matrix(cm, names, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(names))
    plt.xticks(tick_marks, names, rotation=45)
    plt.yticks(tick_marks, names)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')



path = "./data/"

filename = os.path.join(path,"crx.csv")    
df = pd.read_csv(filename,na_values=['NA','?'])

# Encode feature vector
encode_numeric_zscore(df,'a2')
encode_numeric_zscore(df,'s3')
encode_numeric_zscore(df,'a8')
encode_numeric_zscore(df,'a11')
encode_numeric_zscore(df,'a14')
encode_numeric_zscore(df,'a15')
encode_text_dummy(df, 'a1')
encode_text_dummy(df, 'a4')
encode_text_dummy(df, 'a5')
encode_text_dummy(df, 'a6')
encode_text_dummy(df, 'a7')
encode_text_dummy(df, 'a10')
encode_text_dummy(df, 'a12')
encode_text_dummy(df, 'a13')
encode_text_dummy(df, 'a9')
missing_median(df, 'a2')
missing_median(df, 'a14')
a16 = encode_text_index(df,'a16')
num_classes = len(a16)



# Create x & y for training

# Create the x-side (feature vectors) of the training
x, y = to_xy(df,'a16')

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(    
    x, y, test_size=0.25, random_state=42) 

# Create a deep neural network with 3 hidden layers of 10, 20, 5
#classifier = skflow.DNNClassifier(hidden_units=[10, 20, 5], n_classes=num_classes,
#    epochs=10000)
# Create a deep neural network with 3 hidden layers of 10, 20, 5
            feature_columns = [tf.contrib.layers.real_valued_column("", dimension=x.shape[1])]
            self.classifier = learn.DNNClassifier(
                model_dir= self.model_dir,
                config=tf.contrib.learn.RunConfig(save_checkpoints_secs=30),
                hidden_units=self.hidden_units, n_classes=num_classes, feature_columns=feature_columns)

        self.classifier.fit(x,y,steps=self.steps)
        
        return self

# Early stopping
early_stop = skflow.monitors.ValidationMonitor(x_test, y_test,
    early_stopping_rounds=200, print_steps=50, n_classes=num_classes)

# Fit/train neural network
classifier.fit(x_train, y_train, early_stop)

